# Line Business Account 文章資料抓取
 

### 2023/08/31 post_id 換位子
### 2024/02/29 post_id 換位子

### 1. import modules

In [1]:
from selenium import webdriver
import time
# import os
import pandas as pd

import numpy as np
import datetime

from bs4 import BeautifulSoup
import requests

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### 2. 抓文章內容的Function

In [2]:
def func_article_class(bt_link, bt_content_id, bt_title_h1, bt_title_h2, bt_class_main, bt_class_sub, bt_content_date
                       , bt_instroduction, bt_content_all, bt_tag):
    try:
        time.sleep(2)
                
        print(bt_link)
        
        headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
        response = requests.get(bt_link, headers = headers)
        
        print("--1--", response) ## test
        bt_html = BeautifulSoup(response.text, "lxml")

        # --- Begin 抓出文章id，之後會用到 ---
    
        content_id = bt_html.find("meta", {"property":"dable:item_id"}).get("content")
        bt_content_id.append(content_id)

        # --- End 抓出文章id，之後會用到 ---
        
        # --- Begin 大標題 ---
    
        h1_num = 0
        
        title_h1 = bt_html.find_all("h1")  # 搜尋所有h1
        for title in title_h1:
            h1_num += 1
            print("h1:", title.getText())  # 只提取文字部分
            if h1_num == 1:
                
                article_title = title.getText()
                bt_title_h1.append(article_title)
                
                print("article_title", article_title)
        
                if "從今開始，勾勒美好第二人生" in article_title:
                
                    print("幸福熟齡文章")

#                     bt_title_h2.append(bt_link)
#                     bt_class_main.append(bt_link)
#                     bt_class_sub.append(bt_link)
                    bt_content_date.append(bt_link)
                    bt_instroduction.append(bt_link)
                    bt_content_all.append(bt_link)
#                     bt_tag.append(bt_link)

                    continue

        # --- End 大標題 ---

        # --- Begin 次標題 ---
        bt_title_h2_temp = []
        
        title_h2 = bt_html.find_all("h2")  # 搜尋所有h2
        for title in title_h2:
            print("h2:", title.getText())  # 只提取文字部分
            bt_title_h2_temp.append(title.getText())

        # --- End 次標題 ---

        
        # --- Begin 文章分類 ---
        
        ct_class_main = bt_html.find("meta", {"property":"article:section"}).get("content")

        print(bt_html.find("meta", {"property":"article:section"}).get("content"))
        bt_class_main.append(ct_class_main)
        
        ct_class_sub = bt_html.find("meta", {"property":"article:section2"}).get("content")
        
        print(bt_html.find("meta", {"property":"article:section2"}).get("content"))
        bt_class_sub.append(ct_class_sub)
        
        # --- End 文章分類 ---
    
        # --- Begin 文章發佈日 ---

        content_all = bt_html.find_all("div", {"class":"article__wrap"})  # 文章內容

#         lst_content_date = []

        for content in content_all:  # r: 一個li區塊
            ct_date  = content.find("p", {"class":"context__info-item context__info-item--date"})  # 文章發佈日
            bt_content_date.append(ct_date.getText()[:10])

        # --- End 文章分類及文章發佈日 ---

        # --- Begin 前言 ---

#         lst_content_introduction = []

        for content in content_all:
            try:
                ct_introduction = content.find("div", {"class":"article__introduction"})  # 文章簡介
                bt_instroduction.append(ct_introduction.getText())
            except:
                print("No introduction")
                bt_instroduction.append(" ")

        # --- End 前言 ---

        # --- Begin 內文 ---

#         lst_content_all = []

        for content in content_all:  # r: 一個li區塊
        
            ct_all = content.find("div", {"id":f"article__page__{content_id}"})  # 文章內文 (article+page+文章ID)
            bt_content_all.append(ct_all.getText())

        # --- End 內文 ---

        # --- Begin 標籤 ---

#         lst_content_tag = []

        bt_tag_temp = []
        content_tag = bt_html.find_all("a", {"class":"context__tag"})  # 標籤

        for tag in content_tag:  # r: 一個li區塊
            bt_tag_temp.append(tag.getText())

        # --- End 標籤 ---
        
        bt_title_h2_temp = ",".join(bt_title_h2_temp) # 把list中的資料用","join起來
        bt_title_h2.append(bt_title_h2_temp)
#         print("bt_title_h2:", bt_title_h2)
        
        bt_tag_temp = ",".join(bt_tag_temp) # 把list中的資料用","join起來
        bt_tag.append(bt_tag_temp)
#         print("bt_tag:", bt_tag)
        
    except:
        pass
        print("nothing")
        
        bt_content_id.append(bt_link)
        bt_title_h1.append(bt_link)
        bt_title_h2.append(bt_link)
        bt_class_main.append(bt_link)
        bt_class_sub.append(bt_link)
        bt_content_date.append(bt_link)
        bt_instroduction.append(bt_link)
        bt_content_all.append(bt_link)
        bt_tag.append(bt_link)


### 3. 打開網頁

#### 若下方path無法執行，需要至https://chromedriver.chromium.org/downloads 更新「Web_Scraping_in_Python_BeautifulSoup_Selenium&Scrapy_2023/ChromeDriver」內的chromedriver檔案

In [3]:
web = "https://account.line.biz/login?redirectUri=https%3A%2F%2Fmanager.line.biz%2Faccount%2F%40spt0646e%2Finsight%2Ffriends%3Ftab%3Dattribute"
path = 'C:/Users/JoeJian/OneDrive - Business Today Magazine/桌面/Python Codes/Web_Scraping_in_Python_BeautifulSoup_Selenium&Scrapy_2023/ChromeDriver/chromedriver'
## 

driver = webdriver.Chrome(path)

driver.get(web)

driver.maximize_window()

### 4. 登入 Line 帳號

In [1]:
# Enter your Line account
my_email = input("Please enter email: ")

Please enter email: 


In [2]:
# Enter your Line password
my_password = input("Please enter password: ")


Please enter password: 


In [6]:
# Login Line Biz, nedd cellphone double confirm
time.sleep(3)

login_button_1 = driver.find_element_by_xpath("//input[@type='submit']")
login_button_1.click()

# username
time.sleep(2)
login_email = driver.find_element_by_xpath("//input[@name='tid']")
login_email.send_keys(my_email)

time.sleep(2)
login_pwd = driver.find_element_by_xpath("//input[@name='tpasswd']")
login_pwd.send_keys(my_password)

time.sleep(2)
login_button_2 = driver.find_element_by_xpath("//button[@type='submit']")
login_button_2.click()



### 5. 切換到群發訊息

In [7]:
# switch to broadcast pages

broadcast_button = driver.find_element_by_xpath("(//li[@class='nav-item']/a)[4]")
# list中第三個是broadcast的連結
    
broadcast_button.click()

### 6. 輸入欲抓取的資料區間

In [8]:
# search_date = input("Please enter date(YYYY/MM/DD-YYYY/MM/DD): " )
search_date = "2024/02/12-2024/02/15"

In [9]:
search_date_file = search_date.replace("/", "")
search_date_file = search_date_file.replace("-", "_")
print(search_date_file)

20240212_20240215


In [10]:
# select date

date_input = driver.find_element_by_name("datepicker")
date_input.send_keys(search_date)


In [11]:
# enter search button

search_button = driver.find_element_by_xpath("//button[@class='btn btn-primary ml-2']")
search_button.click()


### 7. 抓取資料

In [12]:
# message_all_id = driver.find_elements_by_xpath("//tr[@class='cursor-pointer']/td/div") ## 20230831
message_all = driver.find_elements_by_xpath("//div[@class='dropdown btn-group']") ## 20240229
message_all_id = message_all[3:]

message_id_num = 1
lst_message_id = []
for message in message_all_id:
    
    ### --- 20230831 --- ###
#     if message_id_num % 3 == 0:
#         message_id = message.get_attribute('id')
#         lst_message_id.append(message_id)
        
#     message_id_num += 1
    ### --- 20230831 --- ###
    
    ### --- 20240209 --- ###
    message_id = message.get_attribute('id')
    lst_message_id.append(message_id)

    message_id_num += 1
    ### --- 20240209 --- ###

print(lst_message_id)

['ddown_464630364', 'ddown_464525922', 'ddown_457445384', 'ddown_458326155', 'ddown_458330697', 'ddown_458329774', 'ddown_458325013', 'ddown_458324097']


In [13]:
# get message date and time

# test_id = lst_message_unique_id[0]

df_line_biz_all = pd.DataFrame()

for post_num in range(len(lst_message_id)):
    
    time.sleep(2)
    
    post_id = lst_message_id[post_num]

    print(post_id)
    print("post_num: ", post_num)
    
        
    btn_num = post_num + 1
    print("btn_num:", btn_num)

    menu_post_area = driver.find_elements_by_xpath(f"(//tr[@class='cursor-pointer'])[{btn_num}]/td")
    print("menu_post_area:", menu_post_area)
    
    time.sleep(1)
    
    lst_post_date = []
    lst_post_day = []
    lst_post_time = []
    lst_delivery_tol = []
    lst_open_msg = []
    lst_user_click_cnt = []
    lst_post_date_time = []
    
    for i in range(len(menu_post_area)):

        if i == 0:
            post_date_time = menu_post_area[i].text
            date_time_temp = post_date_time.split('\n')
            print('date_time_temp', date_time_temp)
            lst_post_date.append(date_time_temp[0])
            lst_post_time.append(date_time_temp[1])

            num_day = 0
            for num_day in range(3):
                if num_day == 0:
                    post_year = int(lst_post_date[i].split('/')[num_day])
                    print(post_year)
                elif num_day == 1:
                    post_month = int(lst_post_date[0].split('/')[num_day])
                    print(post_month)
                elif num_day == 2:
                    post_date = int(lst_post_date[0].split('/')[num_day])
                    print(post_date)

                num_day += 1

            today_date = datetime.datetime(post_year, post_month, post_date)
            today_day = today_date.strftime("%w")
            
            if today_day == "0":
                today_day = "7"

            print(today_day)

            lst_post_day.append(today_day)
            
            post_date_time = post_date_time.replace('\n', ' ')
            
            lst_post_date_time.append(post_date_time)

        elif i == 2: ## 已傳送數
            lst_delivery_tol.append(menu_post_area[i].text)
            
            print("傳送數：", menu_post_area[i].text)

        elif i == 3: ## 已開封
            lst_open_msg.append(menu_post_area[i].text)
            
            print("已開封：", menu_post_area[i].text)

        elif i == 4: ## 點擊的用戶
            lst_user_click_cnt.append(menu_post_area[i].text)
            
            print("點擊用戶：", menu_post_area[i].text)

    post_id_num = post_id.split('_')[1]
    
    df_post_status_temp = pd.DataFrame({'id': post_id_num
                                        ,'發文日期': lst_post_date
                                        ,'發文星期': lst_post_day
                                        ,'發文時間': lst_post_time
                                        ,'Post_date_time': lst_post_date_time
                                        ,'已傳送數': lst_delivery_tol
                                        ,'已開封': lst_open_msg
                                        ,'點擊的用戶': lst_user_click_cnt })

#     print(df_post_status_temp)
    
    # click into post: First step, click hole area
    
    time.sleep(0.5)


    post_area_btn = driver.find_element_by_xpath(f"(//tr[@class='cursor-pointer'])[{btn_num}]") # 要帶入第幾個id
    post_area_btn.click()
    
    # click into post: Second step, click "顯示詳情"
    
    ## 2023/08/31 不用點"顯示詳情"
#     time.sleep(0.5)

# #     show_detail_btn_all = driver.find_elements_by_xpath("//button[@class='mb-1 mt-2 btn btn-primary']")
# #     btn_num = post_num + 1
    
#     show_detail_btn = driver.find_element_by_xpath('//a[@class="hljs-link text-primary"]')
    
# #     show_detail_btn = show_detail_btn_all[post_num+1] # 這邊也要調整
    
#     show_detail_btn.click()
    
    # click "顯示更多"
    time.sleep(0.5)

#     show_all_txt_btn = driver.find_element_by_xpath("//a[@class='hljs-link text-primary']")
    show_all_txt_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@class='hljs-link text-primary']")))

    show_all_txt_btn.click()
    
    # block 2 "曝光"

    content_all_ori = driver.find_element_by_xpath("//div[@class='d-flex flex-row justify-content-center']").text

    content_all_ori = content_all_ori.split('\n\n')
    content_i = 0
    content_all = []

    lst_content_title = []
    lst_content_link = []
    lst_post_class = []

    for content_i in range(len(content_all_ori)):
#         content_temp = content_all_ori[content_i].split('\n')
        content_temp = content_all_ori[content_i].split('http')
#         print('content_temp', content_temp) ##
        
        content_temp[0] = content_temp[0].replace(u'\u3000', ' ')
        
#         print('content_temp[0]', content_temp[0]) ##
#         print('content_temp[1]', content_temp[1]) ##
        link_url = 'http' + content_temp[1]
#         print('link_url', link_url)
        
        
        lst_content_title.append(content_temp[0])
        lst_content_link.append(link_url)
        lst_post_class.append(f"文{content_i+1}")


    df_content_temp = pd.DataFrame({'網站標題': lst_content_title
                                   ,'連結網址': lst_content_link
                                   ,'發文類型': lst_post_class})

#     print(df_content_temp)
    
    # block 3 點擊連結

    tbody_all = driver.find_elements_by_xpath("//tbody[@class='text-center']")
    tbody_link_data_all = tbody_all[2].text
    tbody_link_data_all = tbody_link_data_all.split('\n')

    lst_short_link = []
    lst_show_order = []
    lst_impression = []
    lst_click = []
    lst_click_rate = []
    lst_click_user = []
    
    # --- create list for article class ---
    lst_content_id = []
    lst_title_h1 = []
    lst_title_h2 = []
    lst_article_class_main = []
    lst_article_class_sub = []
    lst_content_date = []
    lst_content_introduction = []
    lst_content_all = []
    lst_content_tag = []
    # --- --- --- --- --- --- --- --- --- ---

    for i in range(len(tbody_link_data_all)):
        if i % 2 == 0:
            lst_short_link.append(tbody_link_data_all[i])
            
        else:
            lst_temp_data = tbody_link_data_all[i].split(' ')
            lst_show_order.append(lst_temp_data[0])
            lst_impression.append(lst_temp_data[1])
            lst_click.append(lst_temp_data[2])
            lst_click_rate.append(lst_temp_data[3])
            lst_click_user.append(lst_temp_data[4])
    
    for bt_link in lst_short_link:
        print("bt_link_for:", bt_link)
        func_article_class( bt_link, lst_content_id, lst_title_h1, lst_title_h2, lst_article_class_main, lst_article_class_sub
                          , lst_content_date, lst_content_introduction, lst_content_all, lst_content_tag)# 抓取今周刊官網資料
    
            
    df_link_click_temp = pd.DataFrame({'連結網址': lst_short_link
                                      ,'顯示順序': lst_show_order
                                      ,'曝光': lst_impression
                                      ,'點擊數': lst_click
                                      ,'點擊率': lst_click_rate
                                      ,'點擊的用戶': lst_click_user
                                      ,'文章ID': lst_content_id
                                      ,'文章標題_h1': lst_title_h1
                                      ,'文章標題_h2': lst_title_h2
                                      ,'文章大分類': lst_article_class_main
                                      ,'文章次分類': lst_article_class_sub
                                      ,'文章發布日': lst_content_date
                                      ,'文章前言': lst_content_introduction
                                      ,'文章全文': lst_content_all
                                      ,'文章標籤': lst_content_tag
                                       })

#     print(df_link_click_temp)

    df_content_data_all = pd.merge(df_content_temp, df_link_click_temp, how = 'left')

#     print(df_content_data_all)
    
    df_post_status_dup = pd.concat([df_post_status_temp] * len(df_content_data_all), ignore_index = True)
    
    df_line_biz = pd.concat([df_post_status_dup, df_content_data_all], axis = 1)
    
    df_line_biz_all = pd.concat([df_line_biz_all, df_line_biz], ignore_index = True)
    
    # click 返回一覽

    back_to_previous_page_btn = driver.find_element_by_xpath("//span[@class='text-dark border-dark border-bottom cursor-pointer']")

    back_to_previous_page_btn.click()

print("---------- Finish ------------")

ddown_464630364
post_num:  0
btn_num: 1
menu_post_area: [<selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_124")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_125")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_126")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_127")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_128")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_129")>, <selenium.webdriver.remote.webe

https://bit.ly/49kQ5Na
--1-- <Response [200]>
h1: 開工咖啡優惠一次看！7-11濃粹拿鐵1杯不到23元囤杯訣竅、買2送2週六截止，麥當勞加1元多1件優惠券
article_title 開工咖啡優惠一次看！7-11濃粹拿鐵1杯不到23元囤杯訣竅、買2送2週六截止，麥當勞加1元多1件優惠券
h2: 7-ELEVEn
h2: 全家
h2: 萊爾富
h2: 麥當勞
h2: 星巴克
職場生活
生活消費
bt_link_for: https://bit.ly/4bfVBSh
https://bit.ly/4bfVBSh
--1-- <Response [200]>
h1: 要開工了，還在掙扎不想上班？「收心6部曲」趕走收假症候群，這點最重要…一圖擺脫滿滿厭世感
article_title 要開工了，還在掙扎不想上班？「收心6部曲」趕走收假症候群，這點最重要…一圖擺脫滿滿厭世感
職場生活
健康
bt_link_for: https://bit.ly/4bzzTZP
https://bit.ly/4bzzTZP
--1-- <Response [200]>
h1: 台股開紅盤機率高達75％「2024上看2萬點」！台積電、廣達、聯發科...外資按讚的15檔「龍年必買股」
article_title 台股開紅盤機率高達75％「2024上看2萬點」！台積電、廣達、聯發科...外資按讚的15檔「龍年必買股」
h2: 投資布局，外資多看好三大族群
h2: 外資按讚的15檔「龍年必買股」
投資理財
台股
bt_link_for: 18619
18619
nothing
ddown_458326155
post_num:  3
btn_num: 4
menu_post_area: [<selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae823efc", element="182BADE637137239E68749957772E1EB_element_263")>, <selenium.webdriver.remote.webelement.WebElement (session="178b636dcae3565b4ed3bedfae

bt_link_for: https://bit.ly/3OnBJTD
https://bit.ly/3OnBJTD
--1-- <Response [200]>
h1: 幸福熟齡 - 從今開始，勾勒美好第二人生
article_title 幸福熟齡 - 從今開始，勾勒美好第二人生
幸福熟齡文章
h2: 最新文章
h2: 靚女學
h2: 自我成長
h2: 家庭關係
h2: 退休規劃
h2: 品味生活
h2: 旅讀療心室
h2: 抗老
h2: 更年期
h2: 肌少症
h2: 健康學
h2: 癌症
h2: 慢性病
h2: 失智症
h2: 疑難雜症
h2: 養生
h2: 長期照護
h2: 美好告別
h2: 新冠肺炎
h2: 理財學
h2: 退休理財
h2: 保險規劃
h2: 遺產規劃
h2: People Story
h2: 精彩圓夢
h2: 人生體悟
h2: 名醫養生
h2: 名人私房學
h2: Podcast
h2: Hot影音
h2: 看人物
h2: 學理財
h2: 懂健康
h2: 精選專題活動
h2: 擁抱熟齡必修學分
h2: 熟齡破框 樂活人生
h2: 滿分第二人生
h2: 放膽熟齡
h2: 營養新觀點
h2: 創造健康
h2: 搞懂健檢
h2: 2023幸福熟齡論壇
h2: 最新文章
h2: 靚女學
h2: 自我成長
h2: 家庭關係
h2: 退休規劃
h2: 品味生活
h2: 旅讀療心室
h2: 抗老
h2: 更年期
h2: 肌少症
h2: 健康學
h2: 癌症
h2: 慢性病
h2: 失智症
h2: 疑難雜症
h2: 養生
h2: 長期照護
h2: 美好告別
h2: 新冠肺炎
h2: 理財學
h2: 退休理財
h2: 保險規劃
h2: 遺產規劃
h2: People Story
h2: 精彩圓夢
h2: 人生體悟
h2: 名醫養生
h2: 名人私房學
h2: Podcast
h2: Hot影音
h2: 看人物
h2: 學理財
h2: 懂健康
h2: 精選專題活動
h2: 擁抱熟齡必修學分
h2: 熟齡破框 樂活人生
h2: 滿分第二人生
h2: 放膽熟齡
h2: 營養新觀點
h2: 創造健康
h2: 搞懂健檢
h2: 2023幸福熟齡論壇
h2: 
            到兒女家是作客，「別人家務事」就別管太多！丁菱娟：維持優雅高度不干預，親子皆大歡喜 

In [14]:
df_line_biz_all

,id,發文日期,發文星期,發文時間,Post_date_time,已傳送數,已開封,點擊的用戶,網站標題,連結網址,...,點擊的用戶,文章ID,文章標題_h1,文章標題_h2,文章大分類,文章次分類,文章發布日,文章前言,文章全文,文章標籤
0,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",非常律師陳俊翰離世享年40歲，台大醫證實病因…被判只能活到3-4歲，曾說珍惜每分鐘坦然活著\n,https://bit.ly/3SCXqQV,...,"7,686",202402150044,非常律師陳俊翰離世享年40歲，台大醫證實病因…被判只能活到3-4歲，曾說珍惜每分鐘坦然活著,"被判只能活到3-4歲，重創媽媽但卻從未放棄,六年級大病，曾一度產生放棄念頭：看到媽媽消瘦十分...",產業時事,政治社會,2024-02-15,\n今年初被網路節目《賀瓏夜夜秀》以模仿不雅動作影射的律師陳俊翰，疑似感冒引起併發症、於2/...,\n陳俊翰從小就罹患「先天性脊髓性肌肉萎縮症」（SMA）全身癱瘓，還歷經火警導致雙腿截肢，但...,"陳俊翰離世,陳俊翰,陳俊翰律師,陳俊翰感冒,陳俊翰感染"
1,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",長榮航班孕婦「機上分娩」！作家尪全程接生、乘客目睹現場：剪臍帶嚇傻...懷孕幾週不能搭機\n,https://bit.ly/3SEqaZu,...,"1,635",202402150028,長榮航班孕婦「機上分娩」！作家尪全程接生、乘客目睹現場：剪臍帶嚇傻...懷孕幾週不能搭機,"施景中：適航證明是寫心安的,",職場生活,生活消費,2024-02-15,\n長榮航空週四(2/15)傳出一架從日本羽田飛往台北松山班機，有孕婦在飛機上分娩。對此，民...,\n \n長榮航空說明，羽田-松山BR189班機於飛行途中，一名孕婦出現臨盆反應，經由機上4...,"長榮,孕婦搭飛機,孕婦上飛機,長榮孕婦分娩,空服員,孕婦飛機,長榮航班"
2,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",開工日就宣布熄燈！北投熱海大飯店月底結束營業…曾首創大學溫泉旅宿，學生怎辦？文大回應了\n,https://bit.ly/48kh4XR,...,709,202402150046,開工日就宣布熄燈！北投熱海大飯店月底結束營業…曾首創大學溫泉旅宿，學生怎辦？文大回應了,,職場生活,生活消費,2024-02-15,\n位於台北市北投區、擁有50多年歷史的熱海大飯店，週四(2/15)無預警宣布熄燈，根據官方...,\n \n位於新北投捷運站旁的「北投熱海溫泉大飯店」，在地經營超過50餘年，是許多北投人的共...,"熱海大飯店,熱海大飯店 結束營業,北投,熱海大飯店改建,文化大學"
3,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",海外旅遊被丟包，旅遊不便險會賠？被逼刷卡算「爭議款項」能不繳？金管會給正確答案\n,https://bit.ly/48heh1A,...,615,202402150047,海外旅遊被丟包，旅遊不便險會賠？被逼刷卡算「爭議款項」能不繳？金管會給正確答案,"被逼刷卡才能繼續行程算爭議款項？金管會：需發卡機構認定,海外旅遊被丟包有保險給付？金管會：部...",保險稅制,保險,2024-02-15,\n農曆年節期間，有200位台灣旅客在越南被丟包，許多民眾都相當好奇，遇到這種情況，旅遊不便...,\n對此，金管會在週一（2/15）的例行記者會上說明，依據信用卡定型化契約範本第11條與第1...,"年代旅遊,旅遊不便險"
4,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",「整個市區都引擎聲！」NASA飛機為何繞台灣上空？彭啟明曝1關鍵人物促成：這事要成功很不容易\n,https://bit.ly/3I7dLs1,...,"1,224",202402150030,「整個市區都引擎聲！」NASA飛機為何繞台灣上空？彭啟明曝1關鍵人物促成：這事要成功很不容易,"NASA合作台灣,彭啟明揭背後促成人物：恩師退休前幹大事",職場生活,生活消費,2024-02-15,\n你看到了嗎？美國太空總署(NASA)飛機頻繁飛越台灣上空，不少運氣好的南部民眾紛紛親眼目...,\n \n「哇！快抬頭看上空，NASA飛機在台灣上空繞！」說這話的正是氣象達人、天氣風險公司...,"NASA,NASA飛機,彭啟明,林能暉,環境部"
5,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",一封帛琉總統的信，曝光如何被利誘「只要犧牲台灣」！中國開價買邦交：要訂滿所有飯店房間\n,https://bit.ly/3OIyPsN,...,"1,278",202402150031,一封帛琉總統的信，曝光如何被利誘「只要犧牲台灣」！中國開價買邦交：要訂滿所有飯店房間,,產業時事,政治社會,2024-02-15,\n諾魯共和國與我國斷交，我太平洋友邦帛琉傳出也遭到中國大陸施壓。友邦帛琉總統惠恕仁日前致函...,\n \n社群網路傳出一封由我太平洋友邦帛琉總統惠恕仁給美國參議員的一封信，透露中國向帛琉開...,"帛琉,帛琉旅遊,邦交國,中國,美國"
6,464630364,2024/02/15,4,18:10,2024/02/15 18:10,"123,465","16,539","10,985",「當了20多年兒子，養兒能不能防老我還不知道嗎？」春節裡親戚的對話，道出高房價下年輕人心聲\n,https://bit.ly/42KlaHl,...,"1,521",202402150027,「當了20多年兒子，養兒能不能防老我還不知道嗎？」春節裡親戚的對話，道出高房價下年輕人心聲,,投資理財,房地產,2024-02-15,\n春節就這樣過了！過年期間很多人得到了滿滿的紅包，有人得到的是一身的肥肉，有人則感慨歲月不...,\n \n雖然說年的味道越來越淡了，但過年期間親戚之間的走動是少不了的，因此也就會聽到一些許...,"買房,房價,養兒防老,春節,過年"
7,464525922,2024/02/15,4,12:15,2024/02/15 12:15,"123,461","11,143","3,367",龍年開紅盤台積電攻破7字頭市值18兆、台股飆漲629點史上第2大！現在歷史新高還能買嗎？\n,https://bit.ly/49gF7Ir,...,"1,101",202402150012,龍年開紅盤／外資買超562億史上第5大！台積電攻破7字頭、台股飆漲568點…歷史新高還能買？,"台股點位創新高還能上車？資深分析師：AI第一季見好就收,金龍年台股展望穩健 黃天牧勉業者...",投資理財,台股,2024-02-15,\n台股龍年開紅盤，週一（2/15）上演補漲行情，在權王台積電（2330）早盤創歷史天價、衝...,\n農曆年期間，美國多家重量級企業財報表現亮眼，帶動美股漲勢，與美股連動密切的台股在週一喜迎...,"台股,台積電,台積電股價,台股新高,台積電新高,龍年開紅盤,開紅盤"
8,464525922,2024/02/15,4,12:15,2024/02/15 12:15,"123,461","11,143","3,367",今周重磅》MSCI剔除66家中企，資金出逃流入台股？台積電688學長解套，輝達下周財報能再...,https://bit.ly/49xMWJq,...,381,202402150014,今周重磅》MSCI剔除66家中企，資金出逃流入台股？台積電688學長解套，輝達下周財報能再助攻？,"今周重磅：台股龍年紅盤日 台股指數、台積電股價破紀錄\r\n ,\r\n今周重磅：MSCI剔...",投資理財,台股,2024-02-15,\n台股龍年紅盤日漲勢強勁，盤中創下歷史新高，台積電也衝破700元關卡，同創歷史新高，打破2...,\n今周重磅：台股龍年紅盤日 台股指數、台積電股價破紀錄\r\n \n台股龍年紅盤日一路發，...,"台積電,紅盤日,龍年,MSCI,中概股,台股"
9,464525922,2024/02/15,4,12:15,2024/02/15 12:15,"123,461","11,143","3,367",30年新低！日圓「0.2123」甜甜價，換10萬台幣多賺1萬...日銀何時干預？分析師預測...,https://bit.ly/3weIIrD,...,604,202402150004,30年新低！日圓「0.2123」甜甜價，換10萬台幣多賺1萬...日銀何時干預？分析師預測「...,日圓何時擺脫疲軟？分析師預測時機,投資理財,理財,2024-02-15,\n週四(2/15)台股龍年首個交易日，早盤大漲逾600點，新台幣兌美元匯率以31.35元開...,\n \n新台幣兌美元表現與台股脫鉤，市場分析主要原因為美國1月通膨率回升超出市場預期，增強...,"日

In [15]:
df_line_biz_all[df_line_biz_all["id"].isnull()]

,id,發文日期,發文星期,發文時間,Post_date_time,已傳送數,已開封,點擊的用戶,網站標題,連結網址,...,點擊的用戶,文章ID,文章標題_h1,文章標題_h2,文章大分類,文章次分類,文章發布日,文章前言,文章全文,文章標籤


In [16]:
df_line_biz_all.to_csv(f"Line_biz_{search_date_file}.csv", index = False, encoding = 'UTF-8-sig')

In [17]:
driver.quit()